<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/06_bh_cultura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Início - Bibliotecas**
---

In [1]:
#importando bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
import os
import openpyxl
from google.colab import files
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# **Ver ESTADOS, MUNICÍPIOS e CULTURA**
---

In [3]:
# Escolha de estado e município
url_42 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df42 = pd.read_csv(url_42)

estados = sorted(df42['Estado'].unique())

estado_dropdown = widgets.Dropdown(
    options=estados,
    description='Estado:',
    layout=widgets.Layout(width='250px')
)

municipio_dropdown = widgets.Dropdown(
    description='Município:',
    layout=widgets.Layout(width='350px')
)
def atualizar_municipios(change):
    estado_sel = change['new']
    municipios = sorted(
        df42[df42['Estado'] == estado_sel]['Município'].unique()
    )
    municipio_dropdown.options = municipios

estado_dropdown.observe(atualizar_municipios, names='value')

estado_dropdown.value = estados[0]

display(estado_dropdown, municipio_dropdown)

# ======================================================
# Escolha de cultura
# ======================================================
url_2 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/dados_culturas.csv"
df2 = pd.read_csv(url_2)

cult = sorted(df2['Cultura'].unique())

cult_dropdown = widgets.Dropdown(
    options=cult,
    description='Cultura:',
    layout=widgets.Layout(width='250px')
)

display(cult_dropdown)

Dropdown(description='Estado:', layout=Layout(width='250px'), options=('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF…

Dropdown(description='Município:', layout=Layout(width='350px'), options=(), value=None)

Dropdown(description='Cultura:', layout=Layout(width='250px'), options=('Abacaxi', 'Abóbora', 'Alface', 'Algod…

# **Inserindo informações**
---

In [ ]:
municipio = "Dourados" ###### Nome aqui entre aspas

# ===================================================================
estado = "MS"   # AC, AL, AM, BA, CE, DF, ES, GO, MA, MG, MS, MT,
                # PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO
# ===================================================================

cultura = "Girassol" ## Nome da cultura

ano_inicial = 2023  #Ano da semeadura

#datas de semeadura************************************************************
sem = ['12/11/']  #Datas de semeadura

# **Encontrando a DTA do solo, dados cultura e meteorológicos**
---

In [ ]:
# ============================================================
# Carrega base climática local
# ============================================================
url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Separa colunas categóricas
cidade = df1['Município']
uf = df1['Estado']
regiao = df1['Região']
koppen = df1['Köppen']

# Converte valores com vírgula para ponto e numérico
df1_valores = df1.drop(columns=['Município', 'Estado','Região','Köppen'])
df1_valores = df1_valores.replace({',': '.'}, regex=True)
df1_valores = df1_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df1 = df1_valores.copy()
df1['Município'] = cidade
df1['Estado'] = uf
df1['Região'] = regiao
df1['Köppen'] = koppen

# ============================================================
# Carrega base das culturas
# ============================================================
url_2 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/dados_culturas.csv"
df2 = pd.read_csv(url_2)

# Separa cultura
cult = df2['Cultura']
df2 = df2.drop(columns=['Cultura'])

# Converte texto para numérico
df2_valores = df2.replace({',': '.'}, regex=True)
df2_valores = df2_valores.apply(pd.to_numeric, errors='coerce')
df2 = df2_valores.copy()
df2['Cultura'] = cult

# Filtra a cultura escolhida
df2 = df2[(df2['Cultura'] == cultura)].copy()

# Filtra município e estado
df11 = df1[(df1['Município'] == municipio) & (df1['Estado'] == estado)].copy()

# Variável do solo
DTA = df11['DTA (mm/m)'].iloc[0]

# Duração total da cultura
duracao_do_ciclo = 120  # dias

# % de duração das fases
fase1 = df2['F1 %'].iloc[0]
fase2 = df2['F2 %'].iloc[0]
fase3 = df2['F3 %'].iloc[0]
fase4 = df2['F4 %'].iloc[0]

# Kc nas fases
kc1 = df2['Kc ini'].iloc[0]
kc2 = df2['Kc méd'].iloc[0]
kc3 = df2['Kc fin'].iloc[0]

# Profundidade radicular
z_inicial = 0.05
z_final = df2['Z efetivo (m)'].iloc[0]

# Metadados do local
nome_cidade = df11["Município"].iloc[0]
latitude = df11["latitude"].iloc[0]
longitude = df11["longitude"].iloc[0]
alt = df11["Altitude"].iloc[0]

# ============================================================
# Download de dados NASA POWER
# ============================================================

# Define período de coleta
ano_final = ano_inicial+1
data_ini = datetime(ano_inicial, 1, 1).date()
data_fim = datetime(ano_final, 12, 31).date()

# Formata datas para API
ini = int(data_ini.strftime('%Y%m%d'))
fin = int(data_fim.strftime('%Y%m%d'))

# Monta URL da requisição
base_url = (
    "https://power.larc.nasa.gov/api/temporal/daily/point"
    "?parameters=T2M_MIN,T2M_MAX,PRECTOTCORR,RH2M,WS2M,ALLSKY_SFC_SW_DWN,TOA_SW_DWN"
    "&community=RE"
    "&longitude={longitude}"
    "&latitude={latitude}"
    "&start={ini}"
    "&end={fin}"
    "&format=JSON"
)

api_request_url = base_url.format(
    longitude=longitude,
    latitude=latitude,
    ini=ini,
    fin=fin
)

# Requisição
response = requests.get(url=api_request_url, verify=True, timeout=120)
response.raise_for_status()

# Extração dos parâmetros
dados_json = response.json()
propriedades = dados_json['properties']
parametros = propriedades['parameter']

# Datas
datas_str = sorted(list(parametros['T2M_MIN'].keys()))
datas = [pd.to_datetime(d) for d in datas_str]

# Constrói DataFrame com variáveis
df = pd.DataFrame({'Data': datas})
df['Tmin'] = [parametros['T2M_MIN'][d] for d in datas_str]
df['Tmax'] = [parametros['T2M_MAX'][d] for d in datas_str]
df['Chuva'] = [parametros['PRECTOTCORR'][d] for d in datas_str]
df['UR'] = [parametros['RH2M'][d] for d in datas_str]
df['U2'] = [parametros['WS2M'][d] for d in datas_str]
df['Rs_raw'] = [parametros['ALLSKY_SFC_SW_DWN'][d] for d in datas_str]
df['Qo_raw'] = [parametros['TOA_SW_DWN'][d] for d in datas_str]

# Converte radiação para MJ/m²
df['Rs'] = df['Rs_raw'] * 3.6
df['Qo'] = df['Qo_raw'] * 3.6

# Variáveis intermediárias
df['Tmed'] = (df['Tmax'] + df['Tmin']) / 2
df['Patm'] = 101.3 * ((293 - 0.0065 * alt) / 293) ** 5.26
df['NDA'] = [pd.to_datetime(d).timetuple().tm_yday for d in df['Data']]

# Declinação solar
def dec_sol(NDA):
    return 23.45 * np.sin(np.deg2rad(360/365 * (NDA - 80)))

df['d'] = df['NDA'].apply(dec_sol)

# Hora de nascer do sol
def hora_na_sol(d, Lat):
    return np.rad2deg(np.arccos(-(np.tan(np.deg2rad(Lat)) * np.tan(np.deg2rad(d)))))

df['Hn'] = df['d'].apply(lambda d_: hora_na_sol(d_, latitude))


# (d/D)^2
def relacao_d_D_2(NDA):
    return 1 + 0.033 * np.cos(np.deg2rad(NDA * 360/365))
df['(d/D)²'] = df['NDA'].apply(relacao_d_D_2)

# Balanços radiativos
df['BOC'] = df['Rs'] * 0.75  # (1 - 25%)

df['N'] = df.apply(lambda x: (2 * x['Hn']) / 15, axis = 1)

def e_saturacao(temp):
    return 0.6108 * 10 ** ((7.5 * temp) / (237.3 + temp))

df['es_Tmax'] = df['Tmax'].apply(e_saturacao)
df['es_Tmin'] = df['Tmin'].apply(e_saturacao)
df['es'] = (df['es_Tmax'] + df['es_Tmin']) / 2
df['ea'] = (df['UR']/100) * df['es']
df['QGcs'] = df['Qo'] * (0.75 + (2e-5)*alt)

def BOL(tmax, tmin, eaa, qg, qgcs):
    a = 4.903e-9 * (((tmax+273.16)**4 + (tmin+273.16)**4)/2)
    b = (0.34 - 0.14*np.sqrt(eaa))
    c = 1.35*(qg/qgcs) - 0.35
    return -(a*b*c)

df['BOL'] = df.apply(lambda x: BOL(x['Tmax'],x['Tmin'],x['ea'],x['Rs'],x['QGcs']), axis=1)
df['Rn'] = df['BOC'] + df['BOL']

df['s'] = (4098 * df['es']) / (df['Tmed'] + 237.3)**2
df['gama'] = 0.665e-3 * df['Patm']

# Penman-Monteith
def ETo(s,Rn,gama,u2,es,ea,tmed):
    ETo1 = 0.408*s*Rn
    ETo2 = (gama*900*u2*(es-ea))/(tmed+273)
    ETo3 = s + gama*(1+0.34*u2)
    return (ETo1 + ETo2)/ETo3

df['ETo'] = df.apply(lambda x: ETo(x['s'],x['Rn'],x['gama'],x['U2'],x['es'],x['ea'],x['Tmed']), axis=1)

# Metadados no df
df['Municipio'] = nome_cidade
df['Lat'] = latitude
df['Lon'] = longitude
df['Alt'] = alt

# ============================================================
# Fases fenológicas da cultura
# ============================================================

fasetotal = fase1 + fase2 + fase3 + fase4
fase1d = (duracao_do_ciclo * fase1)/fasetotal
fase2d = (duracao_do_ciclo * fase2)/fasetotal
fase3d = (duracao_do_ciclo * fase3)/fasetotal
fase4d = (duracao_do_ciclo * fase4)/fasetotal

# Incrementos de Kc e Z
acrescimo_kc1 = (kc2 - kc1)/fase2d
acrescimo_kc2 = (kc3 - kc2)/fase4d
acrescimo_z = (z_final - z_inicial)/(fase1d + fase2d)

# ============================================================
# Datas de semeadura e maturação
# ============================================================

dias = [f'{i}{ano_inicial}' for i in sem]
ai = [datetime.strptime(i,'%d/%m/%Y').date() for i in dias]

df1 = pd.DataFrame({'Datas de semeadura': ai})
df1['Datas maturação'] = [i + timedelta(days=duracao_do_ciclo-1) for i in df1['Datas de semeadura']]

df1['Datas de semeadura'] = pd.to_datetime(df1['Datas de semeadura'])
df1['Datas maturação'] = pd.to_datetime(df1['Datas maturação'])



#**Balanço hídrico de cultura**
---

In [ ]:
#**************SIMULAÇÃO DE PRODUTIVIDADE**************************************************
safra1 = (df.loc[(df['Data'] >= df1['Datas de semeadura'][0]) & (df['Data'] <= df1['Datas maturação'][0])]).copy()

safra1['n'] = list(range(1, safra1['NDA'].count() +1))

# -------------------------------
# 1) Kc e FASE da cultura (igual prof)
# -------------------------------
fase_cultura = []
Kc1 = []

for n in safra1['n']:
    if n <= fase1d:
        # Fase 1: Kc = kc1
        Kc1.append(kc1)
        fase_cultura.append(1)

    elif n <= fase1d + fase2d:
        # Fase 2: crescimento linear kc1 -> kc2
        Kc1.append(Kc1[-1] + acrescimo_kc1)
        fase_cultura.append(2)

    elif n <= fase1d + fase2d + fase3d:
        # Fase 3: Kc constante = kc2
        Kc1.append(kc2)
        fase_cultura.append(3)

    else:
        # Fase 4: decaimento linear kc2 -> kc3
        Kc1.append(Kc1[-1] + acrescimo_kc2)
        fase_cultura.append(4)

safra1['Kc'] = Kc1
safra1['Fase'] = fase_cultura

# -------------------------------
# 2) Crescimento da raiz z
# -------------------------------
z1 = []
for n in safra1['n']:
    if n == 1:
        z1.append(z_inicial + acrescimo_z)
    elif n <= fase1d + fase2d:
        z1.append(z1[-1] + acrescimo_z)
    else:
        z1.append(z_final)

safra1['z'] = z1

# -------------------------------
# 3) CAD, ETc, P-ETc
# -------------------------------
safra1['CAD'] = safra1['z'] * DTA          # CAD (mm) no perfil explorado
safra1['ETc'] = safra1['Kc'] * safra1['ETo']    # ETc diária
safra1['P-ETc'] = safra1['Chuva'] - safra1['ETc']

# -------------------------------
# 4) ARM (armazenamento de água no solo)
# -------------------------------
PETc1 = safra1['P-ETc'].to_numpy()
CAD1 = safra1['CAD'].to_numpy()

# Estado inicial de ARM (solo cheio na profundidade inicial)
ARM1 = [z_inicial * DTA]

for p, cad in zip(PETc1, CAD1):
    prev = ARM1[-1]
    if p < 0:
        # Déficit: usa equação exponencial (igual professor)
        ARM1.append(prev * np.exp(p / cad))
    elif p + prev >= cad:
        # Encharcou: limita ao CAD
        ARM1.append(cad)
    else:
        # Situação normal
        ARM1.append(prev + p)

# Remove o estado inicial e deixa só os valores correspondentes aos dias
ARM1 = ARM1[1:]
safra1['ARM'] = ARM1

# -------------------------------
# 5) ALT (variação diária de ARM)
# -------------------------------
ALT = [0]  # primeiro dia, variação zero
ALT += list(np.array(ARM1[1:]) - np.array(ARM1[:-1]))
safra1['ALT'] = ALT

# -------------------------------
# 6) ETR, DEF, EXC, ISNA diário
# -------------------------------
safra1['ETR'] = np.where(
    safra1['P-ETc'] < 0,
    safra1['Chuva'] + safra1['ALT'].abs(),  # solo complementa a chuva
    safra1['ETc']                           # sem limitação hídrica
)

safra1['DEF'] = safra1['ETc'] - safra1['ETR']

safra1['EXC'] = np.where(
    safra1['ARM'] < safra1['CAD'],
    0,
    safra1['P-ETc'] - safra1['ALT']
)

safra1['ISNA_diario'] = safra1['ETR'] / safra1['ETc']

tiempo = 'diario'

#para baixar os dados tire o # da última linha
from google.colab import files
safra1.to_excel(f'{municipio}_{tiempo}.xlsx')
#files.download(f'{municipio}_{tiempo}.xlsx')

safra1

,Data,Tmin,Tmax,Chuva,UR,U2,Rs_raw,Qo_raw,Rs,Qo,...,z,CAD,ETc,P-ETc,ARM,ALT,ETR,DEF,EXC,ISNA_diario
315,2023-11-12,25.14,41.26,0.00,46.67,2.79,7.3999,11.3707,26.63964,40.93452,...,0.056481,4.778333,3.285313,-3.285313,2.126885,0.000000,0.000000,3.285313,0.0,0.000000
316,2023-11-13,25.85,39.89,3.05,50.37,2.46,4.9872,11.3928,17.95392,41.01408,...,0.062963,5.326667,2.517048,0.532952,2.659837,0.532952,2.517048,0.000000,0.0,1.000000
317,2023-11-14,24.77,39.49,1.66,48.95,2.46,5.1533,11.4142,18.55188,41.09112,...,0.069444,5.875000,2.550801,-0.890801,2.285624,-0.374213,2.034213,0.516588,0.0,0.797480
318,2023-11-15,24.57,40.66,0.23,48.12,2.69,6.5369,11.4350,23.53284,41.16600,...,0.075926,6.423333,2.997262,-2.767262,1.485610,-0.800015,1.030015,1.967247,0.0,0.343652
319,2023-11-16,25.26,41.99,0.00,39.43,3.00,6.1260,11.4554,22.05360,41.23944,...,0.082407,6.971667,3.268176,-3.268176,0.929643,-0.555966,0.555966,2.712209,0.0,0.170115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2024-03-06,23.34,34.76,0.41,68.51,0.93,6.1639,10.3138,22.19004,37.12968,...,0.400000,33.840000,4.128624,-3.718624,12.566048,-1.459591,1.869591,2.259033,0.0,0.452836
431,2024-03-07,24.42,34.49,7.45,69.97,1.42,5.2632,10.2691,18.94752,36.96876,...,0.400000,33.840000,3.785600,3.664400,16.230448,3.664400,3.785600,0.000000,0.0,1.000000
432,2024-03-08,24.74,33.19,0.67,72.76,1.75,4.6469,10.2257,16.72884,36.81252,...,0.400000,33.840000,3.408314,-2.738314,14.968823,-1.261625,1.931625,1.476689,0.0,0.566739
433,2024-03-09,24.00,36.96,0.72,67.36,1.72,5.7240,10.1818,20.60640,36.65448,...,0.400000,33.840000,4.156195,-3.436195,13.523476,-1.445347,2.165347,1.990848,0.0,0.520993


# **Resultado por fase**
---

In [ ]:
# -----------------------------------------
# Resumo por fase das principais variáveis
# -----------------------------------------

safra = safra1.groupby("Fase").agg({
    "Tmin": "mean",
    "Tmax": "mean",
    "Tmed": "mean",
    "Chuva": "sum",
    "UR": "mean",
    "ETc": "sum",
    'ETR': "sum",
    "ARM": "last",
    "DEF": "sum",
    "EXC": "sum",
    "ISNA_diario": "mean",
    "N": "mean",
})
safra['Data semeadura'] = df1['Datas de semeadura'].iloc[0]
safra['Data maturação'] = df1['Datas maturação'].iloc[0]
safra = safra.rename(columns={'ISNA_diario': 'ISNA'})
safra = safra.reset_index()

tiempo = 'fase'

#para baixar os dados tire o # da última linha
from google.colab import files
safra1.to_excel(f'{municipio}_{tiempo}.xlsx')
#files.download(f'{municipio}_{tiempo}.xlsx')

safra

,Fase,Tmin,Tmax,Tmed,Chuva,UR,ETc,ETR,ARM,DEF,EXC,ISNA,N,Data semeadura,Data maturação
0,1,24.001000,36.139000,30.070000,88.99,62.802000,43.239469,27.068052,11.847832,16.171417,52.201001,0.735762,13.175605,2023-11-12,2024-03-10
1,2,23.826765,35.855588,29.841176,100.06,65.592353,162.017452,89.411560,15.257709,72.605893,7.238563,0.619795,13.333693,2023-11-12,2024-03-10
2,3,22.816410,36.181795,29.499103,88.57,61.420769,273.444737,100.148445,3.679264,173.296292,0.000000,0.418634,13.070825,2023-11-12,2024-03-10
3,4,23.747407,35.971852,29.859630,86.68,65.523333,136.889362,75.484424,12.016110,61.404938,2.858730,0.588714,12.492074,2023-11-12,2024-03-10
